In [2]:
import requests
import re
import json
import time
from discordwebhook import Discord
import datetime


def getData(persAmmount):
    # Url for the offices other than Ruppertstraße
    url_other = "https://terminvereinbarung.muenchen.de/bba/termin/index.php?loc=BB"
    # Url for Ruppertstraße
    url = "https://www22.muenchen.de/kvr/termin/index.php?cts=1064399=1"
    # Number of appointments for singular person
    einzPersAnz = persAmmount
    # For family
    famAnz = 0
    # Payload
    payload = f"step=WEB_APPOINT_SEARCH_BY_CASETYPES&CASETYPES%5BAn-%20oder%20Ummeldung%20-%20Einzelperson%5D={einzPersAnz}&CASETYPES%5BAn-%20oder%20Ummeldung%20-%20Einzelperson%20mit%20eigenen%20Fahrzeugen%5D=0&CASETYPES%5BAn-%20oder%20Ummeldung%20-%20Familie%5D={famAnz}&CASETYPES%5BAn-%20oder%20Ummeldung%20-%20Familie%20mit%20eigenen%20Fahrzeugen%5D=0&CASETYPES%5BMeldebescheinigung%5D=0&CASETYPES%5BHaushaltsbescheinigung%5D=0&CASETYPES%5BFamilienstands%C3%A4nderung%2F%20Namens%C3%A4nderung%5D=0&CASETYPES%5BAntrag%20Personalausweis%5D=0&CASETYPES%5BAntrag%20Reisepass%2FExpressreisepass%5D=0&CASETYPES%5BAntrag%20vorl%C3%A4ufiger%20Reisepass%5D=0&CASETYPES%5BAntrag%20oder%20Verl%C3%A4ngerung%2FAktualisierung%20Kinderreisepass%5D=0&CASETYPES%5BAusweisdokumente%20-%20Familie%20(Minderj%C3%A4hrige%20und%20deren%20gesetzliche%20Vertreter)%5D=0&CASETYPES%5BeID-Karte%20beantragen%20(EU%2FEWR)%5D=0&CASETYPES%5BNachtr%C3%A4gliche%20Anschriften%C3%A4nderung%20Personalausweis%2FReisepass%2FeAT%5D=0&CASETYPES%5BNachtr%C3%A4gliches%20Einschalten%20eID%20%2F%20Nachtr%C3%A4gliche%20%C3%84nderung%20PIN%5D=0&CASETYPES%5BWiderruf%20der%20Verlust-%20oder%20Diebstahlanzeige%20von%20Personalausweis%20oder%20Reisepass%5D=0&CASETYPES%5BVerlust-%20oder%20Diebstahlanzeige%20von%20Personalausweis%5D=0&CASETYPES%5BVerlust-%20oder%20Diebstahlanzeige%20von%20Reisepass%5D=0&CASETYPES%5BPersonalausweis%2C%20Reisepass%20oder%20eID-Karte%20abholen%5D=0&CASETYPES%5BF%C3%BChrungszeugnis%20beantragen%5D=0&CASETYPES%5BGewerbezentralregisterauskunft%20beantragen%20%E2%80%93%20nat%C3%BCrliche%20Person%5D=0&CASETYPES%5BGewerbezentralregisterauskunft%20beantragen%20%E2%80%93%20juristische%20Person%5D=0&CASETYPES%5BBis%20zu%205%20Beglaubigungen%20Unterschrift%5D=0&CASETYPES%5BBis%20zu%205%20Beglaubigungen%20Dokument%5D=0&CASETYPES%5BBis%20zu%2020%20Beglaubigungen%5D=0&CASETYPES%5BFahrzeug%20wieder%20anmelden%5D=0&CASETYPES%5BFahrzeug%20au%C3%9Fer%20Betrieb%20setzen%5D=0&CASETYPES%5BAdress%C3%A4nderung%20in%20Fahrzeugpapiere%20eintragen%20lassen%5D=0&CASETYPES%5BNamens%C3%A4nderung%20in%20Fahrzeugpapiere%20eintragen%20lassen%5D=0"

    # start session for web requests
    with requests.Session() as s:
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        # get phpsessid
        s.get(url)
        # cookie = s.cookies

        # this is only for getting appointments in the Ruppertstraße
        r = s.post(
            "https://www22.muenchen.de/kvr/termin/index.php?cts=1064399=1",
            data=payload,
            headers=headers,
        )

        rawHTMLRuppert = str(r.content.decode())
        # get phpsessid
        s.get(url_other)
        # cookie = s.cookies
        # this is for getting appointments in the other Offices

        r = s.post(
            "https://terminvereinbarung.muenchen.de/bba/termin/index.php?loc=BB",
            data=payload,
            headers=headers,
        )
        rawHTMLOther = str(r.content.decode())

    # convert to better format
    finderRuppert = re.findall(r"jsonAppoints = .*;", rawHTMLRuppert)[0][16:-2]
    finderOther = re.findall(r"jsonAppoints = .*;", rawHTMLOther)[0][16:-2]

    jDates = dict(json.loads(finderRuppert))
    # Merge together
    jDates.update(dict(json.loads(finderOther)))

    return jDates


# Create webhook object
discord = Discord(url="your_webhook_link")
show_appointment_time = True
preferred_location = None
# Search for bookable dates
found = 0
while found == 0:
    # Refresh the page
    bookables = getData(1)
    # Iterate through each bookable date
    for elem in bookables:
        appointments = bookables[elem]["appoints"]
        name = bookables[elem]["caption"]
        # Check if there is unwanted date (Format: year-month-day)
        not_wanted = ["2022-08-22"]
        wanted = "2022-08"
        for appointment in appointments:
            # Control if the date is desired date
            if (
                len(appointments[appointment]) > 0
                and wanted in appointment
                and appointment not in not_wanted
            ):
                found = 1
                print("FOUNDDD!!", f"Termin am {appointment} reservieren! Ort: {name}")
                if show_appointment_time:
                    print(appointments[appointment])
                discord.post(
                    content=f"Found an appointment - Date: {appointment}, Place: {name} "
                )
    time.sleep(3)


FOUNDDD!! Termin am 2022-08-22 reservieren! Ort: Bürgerbüro Ruppertstraße
FOUNDDD!! Termin am 2022-08-23 reservieren! Ort: Bürgerbüro Ruppertstraße
FOUNDDD!! Termin am 2022-08-24 reservieren! Ort: Bürgerbüro Ruppertstraße
FOUNDDD!! Termin am 2022-08-25 reservieren! Ort: Bürgerbüro Ruppertstraße
FOUNDDD!! Termin am 2022-08-26 reservieren! Ort: Bürgerbüro Ruppertstraße
FOUNDDD!! Termin am 2022-08-22 reservieren! Ort: Bürgerbüro Pasing
FOUNDDD!! Termin am 2022-08-23 reservieren! Ort: Bürgerbüro Pasing
FOUNDDD!! Termin am 2022-08-24 reservieren! Ort: Bürgerbüro Pasing
FOUNDDD!! Termin am 2022-08-25 reservieren! Ort: Bürgerbüro Pasing
FOUNDDD!! Termin am 2022-08-26 reservieren! Ort: Bürgerbüro Pasing
FOUNDDD!! Termin am 2022-08-22 reservieren! Ort: Bürgerbüro Leonrodstraße
FOUNDDD!! Termin am 2022-08-23 reservieren! Ort: Bürgerbüro Leonrodstraße
FOUNDDD!! Termin am 2022-08-24 reservieren! Ort: Bürgerbüro Leonrodstraße
FOUNDDD!! Termin am 2022-08-25 reservieren! Ort: Bürgerbüro Leonrodstraße

In [147]:


privacy

<Response [204]>